In [1]:
# import dependencies
import pandas as pd
import datetime as dt
import pymongo

In [2]:
# load the dataframe

file1=pd.read_csv('Business_Licenses.csv')
# file2=pd.read_csv('Sidewalk_Cafe_Permits.csv')

df1=pd.DataFrame(file1)
# df2=pd.DataFrame(file2)
df1.head()

C:\Users\nilay\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (9,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,ID,LICENSE ID,ACCOUNT NUMBER,SITE NUMBER,LEGAL NAME,DOING BUSINESS AS NAME,ADDRESS,CITY,STATE,ZIP CODE,...,LICENSE TERM START DATE,LICENSE TERM EXPIRATION DATE,LICENSE APPROVED FOR ISSUANCE,DATE ISSUED,LICENSE STATUS,LICENSE STATUS CHANGE DATE,SSA,LATITUDE,LONGITUDE,LOCATION
0,22308-20060816,1723393,29481,1,BELL OIL TERMINAL INC,Bell Oil Terminal LLC,3741 S PULASKI RD 1,CHICAGO,IL,60623,...,08/16/2006,08/15/2007,08/10/2006,08/11/2006,AAI,NaN,NaN,41.825320,-87.723960,"(41.82531992987547, -87.72395999659746)"
1,1620668-20160516,2455262,295026,1,BUCCI BIG & TALL INC.,BUCCI BIG & TALL INC.,558 W ROOSEVELT RD,CHICAGO,IL,60607,...,05/16/2016,05/15/2018,05/18/2016,08/30/2016,AAI,NaN,NaN,41.867339,-87.641590,"(41.86733856638269, -87.64159005699716)"
2,2368602-20160616,2460909,291461,3,"PROJECT: VISION , INC.","PROJECT : VISION , INC",2301 S ARCHER AVE 1 1,CHICAGO,IL,60616,...,06/16/2016,06/15/2017,06/21/2016,06/22/2016,AAC,08/30/2016,NaN,41.850843,-87.638734,"(41.85084294374687, -87.63873424399071)"
3,2060891-20141016,2353257,357247,1,FOLASHADE'S CLEANING SERVICE INC.,FOLASHADE'S CLEANING SERVICE INC.,1965 BERNICE RD 1 1SW,LANSING,IL,60438,...,10/16/2014,10/15/2016,04/01/2016,04/01/2016,AAI,NaN,38.0,41.951316,-87.678586,"(41.95131555606832, -87.67858578019546)"
4,1144216-20070516,1804790,147,63,WALGREEN CO.,Walgreens # 05192,9148 S COMMERCIAL AVE 1ST,CHICAGO,IL,60617,...,05/16/2007,05/15/2008,05/10/2007,05/11/2007,AAI,NaN,5.0,41.728622,-87.551366,"(41.72862173556932, -87.55136646594693)"


In [3]:

df=df1[['ACCOUNT NUMBER','DOING BUSINESS AS NAME','ADDRESS','CITY','STATE','ZIP CODE','LICENSE DESCRIPTION',
       'BUSINESS ACTIVITY ID','BUSINESS ACTIVITY','APPLICATION TYPE','LICENSE TERM START DATE',
       'LICENSE TERM EXPIRATION DATE','DATE ISSUED', 'LICENSE STATUS','LATITUDE', 'LONGITUDE', 'LOCATION']]

#Select the licenses for restaurant and equivalents using the BUSINESS ACTIVITY ID
df=df.loc[(df['BUSINESS ACTIVITY ID']=='775')|
          (df['BUSINESS ACTIVITY ID']=='782') |
         (df['BUSINESS ACTIVITY ID']=='735')]

df['license_id']=df.reset_index().index

In [20]:
df.to_csv('licenses.csv')

In [4]:
#Step 1 to get the list of active licences is to make a column for the year of expiration of the license
#Step 2 is to create a column named 'is_active' with Boolean Values (True if the expiration is 2019 or above, else False)
df['Expiration']=pd.to_datetime(df['LICENSE TERM EXPIRATION DATE']).dt.year
df['is_active']=[True if value>2018 else False for value in df['Expiration']]

In [6]:
# Some business will have two active licenses at the same time because they may renew a license before its term
# expires. So sort the df by the Expiration date. This will mean that those expiring the earlies will be at the bottom,
# and then you can drop the duplicated and the duplicates at the bottom will be dropped
# This ensures that you aren't dropping the most recent licenses (though its not a guarantee)

active_df=df[df['is_active']==True].sort_values('Expiration',
                                                ascending=False).drop_duplicates(subset=['ACCOUNT NUMBER'])
active_df['active_id']=active_df.reset_index().index

In [7]:

active_licenses_table=active_df[['active_id','license_id','LICENSE DESCRIPTION','BUSINESS ACTIVITY ID',
                                 'BUSINESS ACTIVITY','LICENSE TERM START DATE','LICENSE TERM EXPIRATION DATE']]

licenses_table=df[['license_id','LICENSE DESCRIPTION','BUSINESS ACTIVITY ID','BUSINESS ACTIVITY','APPLICATION TYPE',
                    'DATE ISSUED']]

active_locations_table=active_df[['active_id','DOING BUSINESS AS NAME','LATITUDE','LONGITUDE','ADDRESS','CITY','STATE',
                          'ZIP CODE']]

In [8]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [9]:
db = client.restaurantsDB


In [10]:
collection = db.active_licenses
data=active_licenses_table.to_dict(orient='records')
collection.insert_many(data)

In [11]:
collection=db.licenses
data=licenses_table.to_dict(orient='records')
collection.insert_many(data)

In [12]:
collection=db.active_locations
data=active_locations_table.to_dict(orient='records')
collection.insert_many(data)

In [19]:
help(active_locations_table.to_dict)

Help on method to_dict in module pandas.core.frame:

to_dict(orient='dict', into=<class 'dict'>) method of pandas.core.frame.DataFrame instance
    Convert the DataFrame to a dictionary.
    
    The type of the key-value pairs can be customized with the parameters
    (see below).
    
    Parameters
    ----------
    orient : str {'dict', 'list', 'series', 'split', 'records', 'index'}
        Determines the type of the values of the dictionary.
    
        - 'dict' (default) : dict like {column -> {index -> value}}
        - 'list' : dict like {column -> [values]}
        - 'series' : dict like {column -> Series(values)}
        - 'split' : dict like
          {'index' -> [index], 'columns' -> [columns], 'data' -> [values]}
        - 'records' : list like
          [{column -> value}, ... , {column -> value}]
        - 'index' : dict like {index -> {column -> value}}
    
        Abbreviations are allowed. `s` indicates `series` and `sp`
        indicates `split`.
    
    into : c

In [16]:
results = db.active_locations.find()
for result in results:
    print(result)

{'_id': ObjectId('5c6f51e3bd815904ecf9abe2'), 'active_id': 0, 'DOING BUSINESS AS NAME': "GUSI'S PIZZA", 'LATITUDE': 41.702518088000005, 'LONGITUDE': -87.55974505, 'ADDRESS': '10606 S TORRENCE AVE 1', 'CITY': 'CHICAGO', 'STATE': 'IL', 'ZIP CODE': '60617'}
{'_id': ObjectId('5c6f51e3bd815904ecf9abe3'), 'active_id': 1, 'DOING BUSINESS AS NAME': 'THE BRIAR STREET THEATER', 'LATITUDE': 41.938869789, 'LONGITUDE': -87.64908649700001, 'ADDRESS': '3133 N HALSTED ST', 'CITY': 'CHICAGO', 'STATE': 'IL', 'ZIP CODE': '60657'}
{'_id': ObjectId('5c6f51e3bd815904ecf9abe4'), 'active_id': 2, 'DOING BUSINESS AS NAME': 'Phils Pizza of Bridgeport inc.', 'LATITUDE': 41.83075999, 'LONGITUDE': -87.65358378399999, 'ADDRESS': '1102 W 35TH ST', 'CITY': 'CHICAGO', 'STATE': 'IL', 'ZIP CODE': '60609'}
{'_id': ObjectId('5c6f51e3bd815904ecf9abe5'), 'active_id': 3, 'DOING BUSINESS AS NAME': 'SERBIAN EASTERN ORTHODOX', 'LATITUDE': 41.688094405, 'LONGITUDE': -87.532645194, 'ADDRESS': '3737 E 114TH ST 1ST', 'CITY': 'CHICAG

{'_id': ObjectId('5c6f51e3bd815904ecf9b1d8'), 'active_id': 1526, 'DOING BUSINESS AS NAME': 'ROCKY MOUNTAIN CHOCOLATE FACTORY', 'LATITUDE': 42.008536401, 'LONGITUDE': -87.91442843899999, 'ADDRESS': '11601 W TOUHY AVE  T1 B15', 'CITY': 'CHICAGO', 'STATE': 'IL', 'ZIP CODE': '60666'}
{'_id': ObjectId('5c6f51e3bd815904ecf9b1d9'), 'active_id': 1527, 'DOING BUSINESS AS NAME': 'PLENTY CHINESE RESTAURANT', 'LATITUDE': 41.961565583, 'LONGITUDE': -87.679125681, 'ADDRESS': '2002 W MONTROSE AVE 1ST', 'CITY': 'CHICAGO', 'STATE': 'IL', 'ZIP CODE': '60618'}
{'_id': ObjectId('5c6f51e3bd815904ecf9b1da'), 'active_id': 1528, 'DOING BUSINESS AS NAME': 'THE OPTIMA CHICAGO', 'LATITUDE': 41.905433008, 'LONGITUDE': -87.634680857, 'ADDRESS': '1252 N WELLS ST 1-2', 'CITY': 'CHICAGO', 'STATE': 'IL', 'ZIP CODE': '60610'}
{'_id': ObjectId('5c6f51e3bd815904ecf9b1db'), 'active_id': 1529, 'DOING BUSINESS AS NAME': 'CLUB DE NUTRICION UN NUEVO AMANECER', 'LATITUDE': 41.968439079, 'LONGITUDE': -87.72008800299999, 'ADDRES

{'_id': ObjectId('5c6f51e3bd815904ecf9b9a8'), 'active_id': 3526, 'DOING BUSINESS AS NAME': 'HIRO SUSHI AND DESSERT BAR', 'LATITUDE': 41.799427854, 'LONGITUDE': -87.589720489, 'ADDRESS': '1453 E 53RD ST 1ST', 'CITY': 'CHICAGO', 'STATE': 'IL', 'ZIP CODE': 60615.0}
{'_id': ObjectId('5c6f51e3bd815904ecf9b9a9'), 'active_id': 3527, 'DOING BUSINESS AS NAME': 'Wingstop', 'LATITUDE': 41.962234812, 'LONGITUDE': -87.72771371799999, 'ADDRESS': '4445 N PULASKI RD  N', 'CITY': 'CHICAGO', 'STATE': 'IL', 'ZIP CODE': 60630.0}
{'_id': ObjectId('5c6f51e3bd815904ecf9b9aa'), 'active_id': 3528, 'DOING BUSINESS AS NAME': "CANDY'S GROCERY", 'LATITUDE': 41.892599705, 'LONGITUDE': -87.662148696, 'ADDRESS': '1366 W OHIO ST 1', 'CITY': 'CHICAGO', 'STATE': 'IL', 'ZIP CODE': 60642.0}
{'_id': ObjectId('5c6f51e3bd815904ecf9b9ab'), 'active_id': 3529, 'DOING BUSINESS AS NAME': 'BROKEN BARREL', 'LATITUDE': 41.928485478, 'LONGITUDE': -87.663577358, 'ADDRESS': '2548 N SOUTHPORT AVE 1', 'CITY': 'CHICAGO', 'STATE': 'IL', 'Z

{'_id': ObjectId('5c6f51e3bd815904ecf9c177'), 'active_id': 5525, 'DOING BUSINESS AS NAME': '7-ELEVEN #33725', 'LATITUDE': 41.892451801, 'LONGITUDE': -87.62256644899999, 'ADDRESS': '201 E OHIO ST', 'CITY': 'CHICAGO', 'STATE': 'IL', 'ZIP CODE': 60611.0}
{'_id': ObjectId('5c6f51e3bd815904ecf9c178'), 'active_id': 5526, 'DOING BUSINESS AS NAME': 'SIMPLY SOUPS SALADS & SANDWICHES', 'LATITUDE': 41.809372191, 'LONGITUDE': -87.610145899, 'ADDRESS': '635 E 47TH ST', 'CITY': 'CHICAGO', 'STATE': 'IL', 'ZIP CODE': 60653.0}
{'_id': ObjectId('5c6f51e3bd815904ecf9c179'), 'active_id': 5527, 'DOING BUSINESS AS NAME': "CARMINE'S", 'LATITUDE': 41.901864109, 'LONGITUDE': -87.627877452, 'ADDRESS': '1037-1043 N RUSH ST 1ST & 2ND', 'CITY': 'CHICAGO', 'STATE': 'IL', 'ZIP CODE': 60611.0}
{'_id': ObjectId('5c6f51e3bd815904ecf9c17a'), 'active_id': 5528, 'DOING BUSINESS AS NAME': "BENNY'S CHOPHOUSE", 'LATITUDE': 41.890630789, 'LONGITUDE': -87.62682912700001, 'ADDRESS': '444 N WABASH AVE 1ST', 'CITY': 'CHICAGO', 'S

{'_id': ObjectId('5c6f51e3bd815904ecf9c947'), 'active_id': 7525, 'DOING BUSINESS AS NAME': 'Fusion Poke', 'LATITUDE': 41.897740856, 'LONGITUDE': -87.623961316, 'ADDRESS': '835 N MICHIGAN AVE 1ST K4562', 'CITY': 'CHICAGO', 'STATE': 'IL', 'ZIP CODE': '60611'}
{'_id': ObjectId('5c6f51e3bd815904ecf9c948'), 'active_id': 7526, 'DOING BUSINESS AS NAME': 'HAPPINESS RESTAURANT, INC.', 'LATITUDE': 41.833940233, 'LONGITUDE': -87.64603821700001, 'ADDRESS': '3313 S HALSTED ST', 'CITY': 'CHICAGO', 'STATE': 'IL', 'ZIP CODE': '60608'}
{'_id': ObjectId('5c6f51e3bd815904ecf9c949'), 'active_id': 7527, 'DOING BUSINESS AS NAME': 'WE SERVE FOOD MART INC.', 'LATITUDE': 41.69620076, 'LONGITUDE': -87.64235277700001, 'ADDRESS': '10857 S HALSTED ST 1ST', 'CITY': 'CHICAGO', 'STATE': 'IL', 'ZIP CODE': '60628'}
{'_id': ObjectId('5c6f51e3bd815904ecf9c94a'), 'active_id': 7528, 'DOING BUSINESS AS NAME': 'CHINESE SEE THRU KITCHEN', 'LATITUDE': 41.866749383, 'LONGITUDE': -87.668464053, 'ADDRESS': '1651 W ROOSEVELT RD A6